In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from datetime import datetime

In [ ]:
tickers = ["AAPL", "MSFT", "GOOGL", "TSLA", "AMZN"]

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/115.0 Safari/537.36"
}

In [ ]:
def get_stock_data(ticker):
    url = f"https://finance.yahoo.com/quote/{ticker}"
    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Failed to retrieve data for {ticker} (status code: {response.status_code})")
        return None
    soup = BeautifulSoup(response.text, "html.parser")

    price_tag = soup.find("fin-streamer", {"data-field": "regularMarketPrice"})
    change_tag = soup.find("fin-streamer", {"data-field": "regularMarketChange"})
    percent_change_tag = soup.find("fin-streamer", {"data-field": "regularMarketChangePercent"})

    if price_tag and change_tag and percent_change_tag:
        price = price_tag.text.strip()
        change = change_tag.text.strip()
        percent_change = percent_change_tag.text.strip()
        return (price, change, percent_change)
    else:
        print(f"Could not find complete data for {ticker}")
        return None

In [ ]:
while True:
    stock_data = []
    print(f"\nFetching stock data at {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")

    for ticker in tickers:
        data = get_stock_data(ticker)
        if data:
            price, change, percent_change = data
            print(f"{ticker}: ${price} USD | Change: {change} ({percent_change})")
            stock_data.append([ticker, price, change, percent_change])
        else:
            stock_data.append([ticker, "N/A", "N/A", "N/A"])

    filename = f"stock_data_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["Ticker", "Price (USD)", "Change", "Percent Change"])
        writer.writerows(stock_data)

    print(f"\nData saved to {filename}")


    time.sleep(30)